# Merging EB terms-  NLS -  Encyclopaedia Britannica


### Loading the necessary libraries

In [130]:
import yaml
import numpy as np
import collections
import string

In [131]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher

In [132]:
import copy


### Functions

In [133]:
def read_query_results(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    return query_results


In [134]:
def write_query_results(filename, results):
    with open('./results_NLS/'+filename, 'w') as f:
        documents = yaml.dump(results, f)

In [135]:
def create_dataframe(query_results):
  
    
    for edition in query_results:
        for page in query_results[edition]:
            #print(page[1].keys())
            column_list=list(page[1].keys())
            break
        break
        
    data=[]
    for edition in query_results:
        for page in query_results[edition]:
            try:
                data.append(page[1])
               
            except:
                pass
    df = pd.DataFrame(data, columns = column_list)
    #removing the columns that I dont need 
    df= df.drop(['last_term_in_page', 'model', 'num_articles', 'num_page_words', 'num_text_unit' , 'text_unit', 'type_archive'], axis=1)
    #renaming the page num
    df= df.rename(columns={"text_unit_id": "start_page", "type_page": "type_article"})
    #removing 'Page' from the string
    df["start_page"] = df["start_page"].str.replace("Page", "")
    df["start_page"] = df["start_page"].astype(int)
    df["end_page"] = df["end_page"].astype(int)
    df_tmp= df["edition"].str.split("Volume", expand=True)[1].str.split(",", expand=True)
    df["volume"]= df_tmp[0]
    df["letters"] = df_tmp[1]
    df['volume'] = df["volume"].str.replace(" ", "").astype(int)
    df['term'] = df["term"].str.replace("_def", "")
    df['term']= df["term"].str.replace('[^a-zA-Z0-9]', '')
    mask=df["term"].str.isalpha()
    df=df.loc[mask] 
    df['term'] = df['term'].str.upper()
    
    #df['archive'] = df["archive_filename"].str.replace("/ ", "")144133901/
    

    list_editions={"1":["first", "First"], "2":["second", "Second"],\
               "3":["third", "Third"],\
               "4":["fourth", "Fourth"], \
               "5":["fifth","Fifth"], "6":["sixth","Sixth"],\
               "7":["seventh", "Seventh"], "8":["eighth", "Eighth"]} 
    
    for ed in list_editions:
        for ed_versions in list_editions[ed]:
            mask = df["edition"].str.contains(ed_versions)
            df.loc[mask, 'edition_num'] = ed  
    df['edition_num']=df["edition_num"].astype(int)
    a=df["archive_filename"].str.split("/").str[-1]
    df['source_text_file']= a+ "/" + df["source_text_file"]   
    df= df.drop(['edition', 'archive_filename'], axis=1)
    
    
    df = df[["term", "definition", "related_terms", "num_article_words", "header", "start_page", "end_page",  "term_id_in_page", "type_article", "edition_num", "volume", "letters", "year", "title",  "place", "source_text_file"  ]]
    
    df = df[df['term'] != '']
    
    return df

In [136]:
def similar(a, b):
    a=a.lower()
    b=b.lower()
    return SequenceMatcher(None, a, b).ratio()

In [137]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [138]:
def check_string(term, List):
    flag = 0
    for element in List:
        if element in term:
            flag = 1
            break
    if flag == 1:
        return True
    else:
        return False


In [139]:
def clean_topics_terms(term):
    table = str.maketrans('', '', string.ascii_lowercase)
    return term.translate(table)

In [140]:
def create_dataframe_from_file(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    
    df = create_dataframe(query_results)
    return df

In [141]:
def prune_json(json_dict):
    """
    Method that given a JSON object, removes all its empty fields.
    This method simplifies the resultant JSON.
    :param json_dict input JSON file to prune
    :return JSON file removing empty values
    """
    final_dict = {}
    if not (isinstance(json_dict, dict)):
        # Ensure the element provided is a dict
        return json_dict
    else:
        for a, b in json_dict.items():
            if b or isinstance(b, bool):
                if isinstance(b, dict):
                    aux_dict = prune_json(b)
                    if aux_dict:  # Remove empty dicts
                        final_dict[a] = aux_dict
                elif isinstance(b, list):
                    aux_list = list(filter(None, [prune_json(i) for i in b]))
                    if len(aux_list) > 0:  # Remove empty lists
                        final_dict[a] = aux_list
                else:
                    final_dict[a] = b
    return final_dict

In [142]:
def delete_entries(query_results_updated, eliminate_pages):
    new_results={}
    for edition in query_results_updated:
        new_results[edition]=[]
        for page_idx in range(0, len(query_results_updated[edition])):
            if page_idx not in eliminate_pages[edition]:
                new_results[edition].append(query_results_updated[edition][page_idx])
    return new_results

In [143]:
def deleting_adding_entries(query_results_up, eliminate_pages, create_entries):
    new_results={}
    flag = 1
    for edition in query_results_up:
        new_results[edition]=[]
        for page_idx in range(0, len(query_results_up[edition])):
            if page_idx not in eliminate_pages[edition]:
                new_results[edition].append(query_results_up[edition][page_idx])
            else:
                for new_pages in create_entries[edition][page_idx]:
                    new_results[edition].append(new_pages)
            
        
    return new_results      

In [144]:
def related_terms_info(related_terms):
    
    related_data=[]
    for elem in related_terms:
        if elem.isupper() or "." in elem or "," in elem:
            elem=elem.split(".")[0]
            term=elem.split(",")[0]
            if len(term)>2 and term[0].isupper() :
                m = re.search('^([0-9]+)|([IVXLCM]+)\\.?$', term)
                if m is None:
                    term_up = term.upper()
                    if term_up !="FIG" and term_up !="NUMBER" and term_up!="EXAMPLE" and term_up!="PLATE" and term_up!="FIGURE":
                        related_data.append(term_up) and term_up!="EXAMPLE" and term_up!="PLATE" and term_up!="FIGURE"
                        related_data.append(term_up)
    return related_data

In [230]:
def fixing_articles(query_results):
    
    create_entries={}
    eliminate_pages={}
    for edition in query_results:
        create_entries[edition]={}
        eliminate_pages[edition]=[]
        flag_p = 1
        for page_idx in range(0, len(query_results[edition])):
            element = query_results[edition][page_idx][1]
            element_page = query_results[edition][page_idx][0]
            flag = 0
            
            if element["type_page"]=="Topic" and len(element["term"])<=7:
                
                list_terms=[]
                new_entries=[]
                definition=element["definition"]
                definition_list= definition.split(" ")
                term = element["term"]
                flag = 0
                sub_elements=[]
                for word_idx in range(0, len(definition_list)):
                    word = definition_list[word_idx]
                    if word.isupper() and "," in word and len(word)>3 and "See "!= definition_list[word_idx-1] and "SEE " != definition_list[word_idx-1]:
                        sub_elements.append((word.split(",")[0],word_idx))
                        flag = 1
                        
                     
                if flag and len(sub_elements) >= 5:
                    for elem_idx in range(0, len(sub_elements)):
                        term_id = 0
                       
                        new_element={}
                        elem=sub_elements[elem_idx]
                        new_element["term"]=elem[0]
     
                        if elem_idx+1 < len(sub_elements):
                            sentence=definition_list[elem[1]+1: sub_elements[elem_idx+1][1]]
                            new_element["definition"]=' '.join(sentence)
                       
                            
                        else:
                            new_element["last_term_in_page"] = 1
                            try:
                                sentence= definition_list[elem[1]+1:][1]
                                new_element["definition"]=' '.join(sentence)
        
                            except:
                                sentence= definition_list[elem[1]:]
                                if len(sentence) > 3:
                                    new_element["definition"]=' '.join(sentence)
       
                        if "definition" in new_element and len(new_element["term"])>4:
                            
                            new_element["type_page"] = "Article" 
                            new_element["num_article_words"] = len(sentence)  
                            #### related terms ##### 
                            related_terms=[]
                            if "See " in new_element["definition"]:
                                related_terms= new_element["definition"].split("See ")[1]
                            elif "SEE " in new_element["definition"]:
                                related_terms= new_element["definition"].split("SEE ")[1]  
                            new_element["related_terms"]=related_terms_info(related_terms)
                            ####
                            
                            new_element["term_id_in_page"]=term_id 
                            new_element["archive_filename"]= element["archive_filename"]
                            new_element["header"] = element["header"]
                            new_element["model"] = element["model"]
                            new_element["num_page_words"]= element["num_page_words"]
                            new_element["num_text_unit"] = element["num_text_unit"]
                            new_element["place"] = element["place"]
                            new_element["source_text_file"] = element["source_text_file"]
                            new_element["text_unit"] = element["text_unit"]
                            new_element["text_unit_id"] = element["text_unit_id"]
                            new_element["title"] = element["title"]
                            new_element["type_archive"] = element["type_archive"]
                            new_element["year"] = element["year"]
                            new_element["end_page"] =int(element['text_unit_id'].split("Page")[1])
                            new_element["edition"] = element["edition"]
                            
                            new_entries.append(new_element)
                            list_terms.append(new_element["term"])
                            term_id += 1
                        
                if len(list_terms) > 12:
                    for i in new_entries:
                        i["num_articles"] = len(list_terms)
                    eliminate_pages[edition].append(page_idx)
                    create_entries[edition][page_idx]=[]
                    for new_d in new_entries:
                        create_entries[edition][page_idx].append([element_page, new_d])
                             
   
    new_results = deleting_adding_entries(query_results, eliminate_pages, create_entries)
    return new_results
 

In [194]:
def fixing_topics(query_results):
     for edition in query_results:
        for page_idx in range(0, len(query_results[edition])):
            element = query_results[edition][page_idx][1]
            if (element["num_articles"] == 1) and (element["type_page"]=="Article"):
                prev_element = query_results[edition][page_idx-1][1]
                element_term = element["term"]
                if prev_element["type_page"]=="Topic":
                    element["type_page"]=="Topic"
                    element["term"]=prev_element["term"]
                    print("Moved %s to %s" %(element_term, prev_element["term"] ))
    

In [195]:
def merge_articles(query_results):
    eliminate_pages={}
    for edition in query_results:
        eliminate_pages[edition]=[]
        page_number_dict={}
        for page_idx in range(0, len(query_results[edition])):
            
            current_page=query_results[edition][page_idx][0]
            if current_page not in page_number_dict:
                page_number_dict[current_page]=page_idx
            
            element = query_results[edition][page_idx][1]
            if "previous_page" in element['term']:
                current_definition= element["definition"]
                previous_page_idx= page_idx -1
                previous_page_number = current_page -1
                num_article_words=element["num_article_words"]
                related_terms=element["related_terms"]
            
                
                prev_elements = query_results[edition][previous_page_idx][1]
                if prev_elements["last_term_in_page"]:
                   
                    prev_elements["definition"]+=current_definition
                    prev_elements["num_article_words"]+=num_article_words
                    prev_elements["related_terms"]+= related_terms
                    prev_number = int(prev_elements['text_unit_id'].split("Page")[1])
                    prev_elements["end_page"] = current_page
                    
                    for prev_articles_idx in range(page_number_dict[prev_number], page_idx):
                       
                        if query_results[edition][prev_articles_idx][0] == prev_number:
                           
                            query_results[edition][prev_articles_idx][1]["num_page_words"]+=num_article_words
                    
                  
                    for update_element_idx in range(page_number_dict[current_page], page_idx+1):
                        if query_results[edition][update_element_idx][0] == current_page:
                            query_results[edition][update_element_idx][1]["num_page_words"]-=num_article_words
                            query_results[edition][update_element_idx][1]["num_articles"]-=1
                    
                
                eliminate_pages[edition].append(page_idx)
            else:
                element["end_page"] = current_page  
   
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results

In [196]:
def merge_topics(query_results):
    eliminate_pages={}
    provenance_removal={}
    freq_topics_terms={}
    merged_topics={}
    parts_string=["Part", "Fart", "Parc", "CPart", "PI"]
    for edition in query_results:
        eliminate_pages[edition]=[]
        provenance_removal[edition]=[]
        freq_topics_terms[edition]={}
        merged_topics[edition]={}
        
        page_idx = 0
        while page_idx < len(query_results[edition]):
            current_page=query_results[edition][page_idx][0]        
            element = query_results[edition][page_idx][1]

            if "Topic" in element['type_page'] and element["term"]!="" and element["term"]!=" ":
                
                if check_string(element["term"], parts_string):
                   
                    ### It means that the previous page was a topic
                    ### And we have one before and correct the error
                    page_idx = page_idx -1 
                    element = query_results[edition][page_idx][1]
                    element['type_page']="Topic"
                    
            
                term=element["term"]
                clean_term=clean_topics_terms(term)
                
                next_page_idx= page_idx + 1
                      
                if next_page_idx < len(query_results[edition]):
                    flag=0
                    tmp_idx = 0
                    for p_id in range(next_page_idx, len(query_results[edition])):
                        next_element = query_results[edition][p_id][1]
   
                        if not check_string(next_element["term"], parts_string):
                            next_term=clean_topics_terms(next_element["term"])
                        else:
                            next_term=next_element["term"]
                    
                        if similar(clean_term, next_term) > 0.72 or check_string(next_term, parts_string) or next_term in clean_term: 

                            if term not in merged_topics[edition]:
                                merged_topics[edition][clean_term]=[]
                        
                            if not check_string(next_term, parts_string) :
                                 merged_topics[edition][clean_term].append(next_term)
                         
                            element["definition"]+=next_element["definition"]
                            element["num_article_words"]+=next_element["num_article_words"]
                            element["num_page_words"]+=next_element["num_page_words"]                  
                            element["related_terms"]+= next_element["related_terms"]
                            element["end_page"] = int(next_element['text_unit_id'].split("Page")[1])
                            provenance_removal[edition].append(element["end_page"])

                            eliminate_pages[edition].append(p_id)
                            tmp_idx= p_id + 1
                        else:
                            flag = 1
                            break
                  
        
                    if flag:
                         page_idx= p_id
                    
                    else:
                        page_idx = tmp_idx
                        
                 
                    if clean_term in merged_topics[edition]:
                       
                        if merged_topics[edition][clean_term]:
                            freq_term=most_frequent(merged_topics[edition][clean_term])
                            freq_topics_terms[edition][term]=freq_term
                            element["term"]=freq_term
                        else:
                            element["term"]=clean_term
                    else:
                        element["term"]=clean_term
                    
                else:
                    page_idx = next_page_idx
                    
                
               
            else:
                page_idx += 1
           
    #for ed in provenance_removal:
    #    print("ED:%s -- removing the following pages %s" %(ed, provenance_removal[ed]))
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results, merged_topics, freq_topics_terms

In [197]:
def merge_topics_refine(query_results):
    
    topics_editions={}
    eliminate_pages={}
    merged_topics_refine={}
    provenance_removal={}
    for edition in query_results:
        eliminate_pages[edition]=[]
        provenance_removal[edition]=[]
        topics_editions[edition]={}
        merged_topics_refine[edition]=[]
        page_idx = 0
        character="A"
        while page_idx < len(query_results[edition]):
            
            element = query_results[edition][page_idx][1]
            term = element["term"]
           
            
            if "Topic" in element['type_page'] and term!="" and term!=" " and term[0] >= character:
                character=term[0]
                if term not in topics_editions[edition]:
                    topics_editions[edition][term]=page_idx
                    #print("NEW: Topic --%s-- - Page: %s "%(term, topics_editions[edition][term]))
                else:
                    start_page_idx=topics_editions[edition][term]
                    first_element = query_results[edition][start_page_idx][1]
                    
                    first_element["definition"]+=element["definition"]
                    first_element["num_article_words"]+=element["num_article_words"]
                    first_element["num_page_words"]+=element["num_page_words"]                  
                    first_element["related_terms"]+= element["related_terms"]
                    first_element["end_page"] = int(element['text_unit_id'].split("Page")[1])
                    provenance_removal[edition].append(first_element["end_page"])
                    merged_topics_refine[edition].append(term)
                    eliminate_pages[edition].append(page_idx)
                    
                
            
            page_idx += 1
            if term:
                character=term[0]
        
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results, provenance_removal, merged_topics_refine

### 1. Reading data

Here we are going to take the output of the defoe files, and we are going to merge the terms that splitted across pages. 

The next line takes time!

In [48]:
query_results=read_query_results('results_eb_1_edition')

In [233]:
dc_results = copy.deepcopy(query_results)

### 2. Merging articles splitted across pages

Articles normaly have a lenght of one or two paragraphs, but they could be splitted across two consecutive pages.
So, we are going to merge the detected splitted articles into one. 

In [234]:
query_results_articles = merge_articles(dc_results)

Since some pages with articles could have wronlgy been classified as "topics", we are going to go through the "TOPICS" pages, and try to extract again articles from those again - reclassifying those as Articles. 

In [235]:
articles_refined=fixing_articles(query_results_articles)

### 3. Merging topics together across pages 

First, we are going to transfor those pages, which only have 1 article, and the previous term is a topic, into topics, and give the previous term to these ones.

In [236]:
fixing_topics(articles_refined)

Moved first_article to ALGEBRA
Moved first_article to ANATOMY
Moved first_article to RITHMETICK
Moved first_article to ARITHMETICK
Moved first_article to WASTE-BOOK
Moved first_article to BOOKKEEPING
Moved first_article to BOOK-KEEPING
Moved first_article to METAPHYSICS
Moved first_article to ilate
Moved first_article to aPNEUMATICS
Moved first_article to SHORTHANDWRITING
Moved first_article to ARITHMETICK
Moved 31S to ARITHMARITHM
Moved first_article to oARITHM
Moved first_article to BOOKK
Moved first_article to BOOKKEEPING
Moved first_article to BREWNG
Moved first_article to CHEMstry
Moved first_article to CHEMISTRY
Moved first_article to FAIIRIERY
Moved first_article to MEDICINE
Moved first_article to MEDICINE
Moved first_article to MEDICINE
Moved first_article to MIDWIFERY
Moved first_article to NAVIOATION


Now, we are goint to merge topics.
Topics can be across several pages, so we are going to **merge consecutive page-topics together**.


In [237]:
topics_refined, merged_topics, freq_topics_terms =merge_topics(articles_refined)

In [238]:
#for ed in merged_topics:
#    print("ED: %s, meged_topics %s" %(ed, merged_topics[ed]))

In [239]:
#for ed in freq_topics_terms:
#    print("ED: %s, freq_topics_terms %s" %(ed, freq_topics_terms[ed]))

#### 3.1 Merging topics even more

Before, we were merging topics that were in consecutive pages. But very often we have other types of pages (with tables, figures) between the same topic. Therefore we are going to merge the pages with the same topics. 

In [240]:
final_refine, provenance_removal,merged_topics_refine =merge_topics_refine(topics_refined)

In [241]:
for ed in merged_topics_refine:
    print("ED: %s, merged_topics_refine %s" %(ed, merged_topics_refine[ed]))

ED: First edition, 1771, Volume 1, A-B, merged_topics_refine ['ALGEBRA', 'ALGEBRA', 'ANATOMY', 'ARCHITECTURE', 'ARCHITECTURE', 'ARITHMETICK', 'ARITHMETICK', 'ASTRONOMY', 'ASTRONOMY', 'BOTANY', 'BOTANY']
ED: First edition, 1771, Volume 2, C-L, merged_topics_refine ['CHEMISTRY', 'CHEMISTRY', 'FARRIERY', 'SFLUXIONS', 'GEOMETRY', 'GEOMETRY', 'GRAMMAR', 'GRAMMAR', 'LA', 'LA', 'LLW', 'LAW', 'LAW', 'LA', 'LAW', 'LAW', 'LAW']
ED: First edition, 1771, Volume 3, M-Z, merged_topics_refine ['MEDICINE', 'METAPHYSICS', 'METAPHYSICS', 'MIDWIFERY', 'MIDWIFERY', 'NAVIGATION', 'OPTICS', 'I', 'OPTICS', 'SHORTHANDWRITING', 'SURGERY', 'SURGERY', 'SURGERY', 'SURGEY', 'SURGERY', 'TRIGONOMETRY', 'TRIGONOMETRY']
ED: First edition, 1773, Volume 1, A-B, merged_topics_refine ['ARITHMETICS', 'ARITHM', 'ARITHMETICK', 'P']
ED: First edition, 1773, Volume 2, C-L, merged_topics_refine ['CHEMISTRY', 'CHEMISTRY', 'CHEMICHEMI', 'CHEMISTRY', 'FARRIERY', 'FARRIERY', 'FLUXIONS', 'GARDENING', 'GARDENING', 'GEOMETRY', 'GRAMMA

Once the data has been merged, we are going to store it in a file, just to have the data merged.  

In [249]:
write_query_results("results_eb_1_edition_updated", final_refine)

### 2. Creating a dataframe from the updated results

Once, we have the terms properly merged, we are going to create a dataframe, which we will be use later to do further exploration. In this dataframe we have dropped some information from the original defoe files, that we don not longer need. 

**The dataframe will have the following columns**

- definition:           Definition of the article
- edition_num:          1,2,3,4,5,6,7,8
- header:               Header of the page's article                                  
- num_article_words:    Number of words per article
- place:                Place where the volume was edited (e.g. Edinburgh)                                    
- related_terms:        Related articles (see X article)  
- source_text_file:     File Path of the XML file from which the article belongs       
- term:                 Article name                            
- term_id_in_page:      Number of article in the page     
- start_page:           Number page in which the article starts 
- end_page:             Number page in which the article ends 
- title:               Title of the Volume
- type_article:            Type of Page [Full Page| Topic| Mix | Articles]                                       
- year:                 Year of the Volume
- volume:               volume (e.g. 1)
- letters:              leters of the volume (A-B)


### IMPORTANT DECISION

I am going to filter OUT all the entries which are not Articles, Topics, or Mix.

In [242]:
df=create_dataframe(final_refine)

In [243]:
df

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,volume,letters,year,title,place,source_text_file
0,IIIUBNSFVNSFV,n*s-f 7^' v L i A j J ^ /^^W / ; H:;^’ J }r-r£...,[],58,iiiubnsfvnsfv,2,3,0,FullPage,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082735.34.xml
1,FIRSTARTICLE,S :u -I >;J .1 M U a C V',[],10,**■*,8,8,0,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082813.34.xml
2,FRONTPAGE,COVI PILED UPON A NEW PLAN. IN WHICH The difer...,[],91,FrontPage,9,9,0,FullPage,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082826.34.xml
3,PREFACE,UTILITY ought to be the principal intention of...,[],363,Preface,11,11,0,FullPage,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082852.34.xml
4,PREFACE,"TH E Editors, though fully fen&ble of the prop...",[],410,viPREFACE,12,12,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082865.34.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27862,ZWEIBRUGGEN,"a county of the palatinate of the Rhine, in Ge...",[SQALVS],23,ZoDZYG,857,857,23,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188375020.34.xml
27863,ZYGOMA,in anatomy. See Anatomy p. 152.,[],6,ZoDZYG,857,857,24,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188375020.34.xml
27864,ZYGOMATICUS,"in anatomy,. See Anatomy, p. 306,",[ANATOMY],6,ZoDZYG,857,857,25,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188375020.34.xml
27865,ZYGOPHYLLUM,"in botany, a gr.nus of the decanoria monogyma ...",[],48,ZoDZYG,857,857,26,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188375020.34.xml


In [244]:
includeKeywords=["Article", "Topic", "Mix"]
df=df[df["type_article"].str.contains('|'.join(includeKeywords))]

In [245]:
df.head(n=20)

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,volume,letters,year,title,place,source_text_file
1,FIRSTARTICLE,S :u -I >;J .1 M U a C V',[],10,**■*,8,8,0,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082813.34.xml
4,PREFACE,"TH E Editors, though fully fen&ble of the prop...",[],410,viPREFACE,12,12,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082865.34.xml
5,OR,"Albini tabule anatomies, Alfton’s ‘Tirocinium ...",[],519,LISTofAUTHORSc,13,15,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082878.34.xml
6,AABAM,"a term, among alchemifts, for lead,",[],6,EncyclopaediaBritannica,15,15,1,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
7,AACH,the name of a town and river in Swabia. It is ...,[],17,EncyclopaediaBritannica,15,15,2,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
8,AADE,"the name of two rivers, one in the country of ...",[],19,EncyclopaediaBritannica,15,15,3,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
9,AAHUS,a small town and diftrift in Weftphalia.,[],7,EncyclopaediaBritannica,15,15,4,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
10,AAM,"a Dutch measure for liquids, containing about ...",[],17,EncyclopaediaBritannica,15,15,5,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
11,AAR,"the name of two rivers, one in Weftphalia, and...",[],25,EncyclopaediaBritannica,15,15,6,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml
12,AARSEO,"a town in Africa, situated near the mouth of",[],9,EncyclopaediaBritannica,15,15,7,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082904.34.xml


### 3. Saving the dataframe to json file 

In [250]:
df.to_json(r'./results_NLS/results_eb_1_edition_postprocess_dataframe', orient="index") 

In [247]:
df[df["type_article"]=="Topic"]

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,volume,letters,year,title,place,source_text_file
4,PREFACE,"TH E Editors, though fully fen&ble of the prop...",[],410,viPREFACE,12,12,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082865.34.xml
5,OR,"Albini tabule anatomies, Alfton’s ‘Tirocinium ...",[],519,LISTofAUTHORSc,13,15,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082878.34.xml
141,ABRABR,"(6 ABRASA, in surgery, ulcers, where the Ikin ...","[RENUNCIATION, ABRIDGEMENT, BUT, BUT, BUT, MIR...",1451,ABRABR,20,21,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082969.34.xml
834,AGRICULTURE,"The outer coat is extremely thin, and full of ...","[AQUIFOLIUM, AGRIMONIA, AGRIMONY, BRITAIN, AGR...",31721,AGRICULTURE,61,100,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188083505.34.xml
1081,BALGEBRA,been imagined for representing their affefitio...,[],1315,BoALGEBRA,112,113,0,Topic,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188084168.34.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27134,TRICONIJ,"is to the leg BG 87 I-939J 2 so is radius 90°,...",[],1257,TRICONIJ,807,808,0,Topic,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188374370.34.xml
27140,TRICON,P Is to their difference 21 X.32222 I So is th...,[],1083,TRICON,809,809,0,Topic,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188374396.34.xml
27617,WATCHLOCKWORK,the wheel D to turn along with the barrel ; an...,"[MEDICINE, WATER]",2536,WATCHandCLOCKWORK,837,838,0,Topic,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188374760.34.xml
27618,PCPX,"Fit/. -3. . //},//./,, ''WX-T CH and Cl O CK W...",[],97,PlateCPX,839,841,0,Topic,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850368/alto/188374786.34.xml


In [248]:
df[df["term"]=="ABERDEEN"]

,term,definition,related_terms,num_article_words,header,start_page,end_page,term_id_in_page,type_article,edition_num,volume,letters,year,title,place,source_text_file
89,ABERDEEN,"the name of two cities in Scotland, called the...",[],415,ABE,18,18,0,Article,1,1,A-B,1771,"Encyclopaedia Britannica; or, A dictionary of ...",Edinburgh,144133901/alto/188082943.34.xml
13898,ABERDEEN,"the name of two cities in Scotland, called the...",[],413,ABEABI,18,18,1,Article,1,1,A-B,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,144850366/alto/188283831.34.xml
